Here are metrics we'd like to understand from Retro Funding 4:

- OP spent per ETH in gas fees rewarded
- OP spent per ETH in Collective Contributions rewarded
- OP spent per Trusted user
- OP spent per active address

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/private/rf4_results.csv')
df.tail(1)

,application_id,project_name,is_oss,gas_fees,transaction_count,trusted_transaction_count,trusted_transaction_share,trusted_users_onboarded,daily_active_addresses,trusted_daily_active_users,...,0x94db037207f6fb697dbd33524aadffd108819dc8,0xbc39fb41fe0229352774930c5aa3bf1635c2665f,0x34aa3f359a9d614239015126635ce7732c18fdf3,0x0331969e189d63fbc31d771bb04ab44227d748d8,0xcf79c7eaec5bdc1a9e32d099c5d6bdf67e4cf6e8,0x7f22646e85da79953ac09e9b45ee1b3be3a42abc,0x69dc230b06a15796e3f42baf706e0e55d4d5eaa1,0xebb03f5a580ff088bd4269f88a0cfe5eaf00d2b0,median,rf4_allocation
229,0x1c1bb58707493e944e721acf773ee77405f8eb4c8207...,Building Blocks,False,0.006829,1266.0,213.0,0.168246,13.0,3.733607,0.770492,...,143.784931,6977.138582,4305.014275,5627.27387,2813.695117,821.410629,720.176794,6681.501257,857.363532,0.0


In [3]:
rewards = df['rf4_allocation']
gas_fees = df['gas_fees']
addresses = df['monthly_active_addresses']
gas_fees.sum()

2367.5344501573304

In [4]:
query = """
    select
      events.event_source,
      count(distinct events.from_artifact_name) as num_address,
      count(distinct events.trusted_user_id) as unique_users,
      sum(events.amount / 1e18) as amount
    from `oso.rf4_events_daily_to_project` as events
    join `oso.projects_v1` as projects_v1
      on events.project_id = projects_v1.project_id
    join `static_data_sources.rf4_project_eligibility` as rf4_project_eligibility
      on projects_v1.project_name = rf4_project_eligibility.oso_name
    where
      rf4_project_eligibility.is_approved is true
      and events.event_type = 'CONTRACT_INVOCATION_DAILY_L2_GAS_USED'
      and events.bucket_day >= '2023-10-01'
    group by 1
"""
results = [{
  "event_source": "OPTIMISM",
  "num_address": "2083101",
  "unique_users": "51303",
  "amount": "763.53973658976042"
}, {
  "event_source": "MODE",
  "num_address": "188101",
  "unique_users": "25801",
  "amount": "184.009741333596"
}, {
  "event_source": "BASE",
  "num_address": "2557075",
  "unique_users": "65238",
  "amount": "1276.4211504319107"
}, {
  "event_source": "ZORA",
  "num_address": "734583",
  "unique_users": "38939",
  "amount": "191.54608365319274"
}, {
  "event_source": "FRAX",
  "num_address": "156",
  "unique_users": "14",
  "amount": "0.032961275915125945"
}]

In [5]:
df_results = pd.DataFrame(results)
df_results['amount'] = pd.to_numeric(df_results['amount'])

In [7]:
df_results['amount'].sum()

2415.5496732843753

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns